#### Install required libraries

#### Install required libraries

In [1]:
! pip install git+https://github.com/TorkamaniLab/lohrasb.git --force-reinstall
! pip install pandas interpret category_encoders feature-engine catboost


  Cloning https://github.com/TorkamaniLab/lohrasb.git to /private/var/folders/v1/xbcjnd1x5rn7ct1m_rnsblk80000gp/T/pip-req-build-kl83wzd7
  Running command git clone --filter=blob:none --quiet https://github.com/TorkamaniLab/lohrasb.git /private/var/folders/v1/xbcjnd1x5rn7ct1m_rnsblk80000gp/T/pip-req-build-kl83wzd7
  Resolved https://github.com/TorkamaniLab/lohrasb.git to commit c80394969eff33bf707e16933e5da11b20942914
  Preparing metadata (setup.py) ... done
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Obtaining dependency information for alembic from https://files.pythonhosted.org/packages/a2/8b/46919127496036c8e990b2b236454a0d8655fd46e1df2fd35610a9cbc842/alembic-1.12.0-py3-none-any.whl.metadata
  Using cached alembic-1.12.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached ansi2html-1.8.0-py3-none-any.whl (16 kB)
  Using cached appnope-0.1.3-py2.py3-none-any.whl (4.4 kB)
  Obtaining dependency information for argcomplete from https://files.pythonhosted.org/packages/4f/e

#### Import and print some versions

In [2]:
# Standard library imports
import sys  # For system-related operations like fetching Python version

# Third-party library imports
import numpy as np  # Aliased for better readability
import pandas as pd  # Aliased for better readability
import sklearn  # For machine learning utilities
import optuna  # For hyperparameter optimization
import catboost  # For CatBoost classifier

# Scikit-learn specific imports
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, make_scorer
from sklearn.pipeline import Pipeline

# Feature engineering imports
from feature_engine.imputation import MeanMedianImputer

# Optuna specific imports
from optuna.pruners import HyperbandPruner
from optuna.samplers._tpe.sampler import TPESampler

# Catboost specific imports
from catboost import CatBoostClassifier

# Local (or application-specific) imports
import lohrasb
from lohrasb.best_estimator import BaseModel

# Print out versions of key libraries
print(f'Python version : {sys.version}')
print(f'lohrasb version : {lohrasb.__version__}')
print(f'sklearn version : {sklearn.__version__}')
print(f'pandas version : {pd.__version__}')  # Using the alias
print(f'numpy version : {np.__version__}')
print(f'catboost version : {catboost.__version__}')
print(f'optuna version : {optuna.__version__}')


/Users/hjavedani/.pyenv/versions/3.7.8/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Errno 2] No such file or directory: '/Users/hjavedani/.pyenv/versions/3.7.8/lib/python3.7/site-packages/lohrasb/config.yaml'
In this module, the default logging will be applied. The error is [Errno 2] No such file or directory: '/Users/hjavedani/.pyenv/versions/3.7.8/lib/python3.7/site-packages/lohrasb/config.yaml' which will be skipped!
default logger setting is applied !


2023-09-09 09:39:27,846	INFO util.py:90 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-09-09 09:39:28,453	INFO util.py:90 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Python version : 3.7.8 (default, Feb 27 2023, 18:11:31) 
[Clang 14.0.0 (clang-1400.0.29.202)]
lohrasb version : 4.2.0
sklearn version : 1.0.2
pandas version : 1.3.5
numpy version : 1.21.6
catboost version : 1.2.1
optuna version : 3.3.0


#### Example: Audiology (Standardized) Data Set
#### https://archive.ics.uci.edu/ml/datasets/Audiology+%28Standardized%29


In [3]:
urldata = "https://archive.ics.uci.edu/ml/machine-learning-databases/lymphography/lymphography.data"
urlname = "https://archive.ics.uci.edu/ml/machine-learning-databases/lung-cancer/lung-cancer.names"
# column names
col_names = [
    "class",
    "lymphatics",
    "block of affere",
    "bl. of lymph. c",
    "bl. of lymph. s",
    "by pass",
    "extravasates",
    "regeneration of",
    "early uptake in",
    "lym.nodes dimin",
    "lym.nodes enlar",
    "changes in lym.",
    "defect in node",
    "changes in node",
    "special forms",
    "dislocation of",
    "exclusion of no",
    "no. of nodes in",

]
data = pd.read_csv(urldata,names=col_names)
data.head()

,class,lymphatics,block of affere,bl. of lymph. c,bl. of lymph. s,by pass,extravasates,regeneration of,early uptake in,lym.nodes dimin,lym.nodes enlar,changes in lym.,defect in node,changes in node,special forms,dislocation of,exclusion of no,no. of nodes in
3,4,2,1,1,1,1,1,2,1,2,2,2,4,8,1,1,2,2
2,3,2,1,1,2,2,1,2,1,3,3,2,3,4,2,2,2,2
3,3,2,2,2,2,2,2,2,1,4,3,3,4,8,3,2,2,7
3,3,1,1,1,1,2,1,2,1,3,3,4,4,4,3,1,2,6
2,3,1,1,1,1,1,1,1,1,2,2,4,3,5,1,2,2,1


#### Define labels and train-test split


In [4]:

data.loc[(data["class"] == 1) | (data["class"] == 2), "class"] = 0
data.loc[data["class"] == 3, "class"] = 1
data.loc[data["class"] == 4, "class"] = 2
data["class"] = data["class"].astype(int)

# Train test split

X = data.loc[:, data.columns != "class"]
y = data.loc[:, data.columns == "class"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33,  random_state=42
)



#### Define estimator and set its arguments 


In [5]:
estimator = CatBoostClassifier()
estimator_params = {
        "depth": [4, 5],
        "learning_rate": [0.01, 0.1],

    }

kwargs = {  
            # extra params for model if any
            'main_random_kwargs':{},
            # params for fit method or fit_params 
            'fit_grid_kwargs' :{
            'sample_weight':None,
            },
            # params for RandomSearchCV 
            'random_search_kwargs' : {
            'estimator':estimator,
            'param_distributions':estimator_params,
            'scoring' :'f1_weighted',
            'verbose':3,
            'n_jobs':-1,
            'cv':KFold(2),
            'refit':True,
            }
            }
    

#### Define BestModel using random search

In [6]:

obj = BaseModel().optimize_by_randomsearchcv(
    kwargs=kwargs
        )

#### Capture int features (the problem has only integer features)

In [7]:
int_cols = X_train.select_dtypes(include=["int"]).columns.tolist()
print(int_cols)

['lymphatics', 'block of affere', 'bl. of lymph. c', 'bl. of lymph. s', 'by pass', 'extravasates', 'regeneration of', 'early uptake in', 'lym.nodes dimin', 'lym.nodes enlar', 'changes in lym.', 'defect in node', 'changes in node', 'special forms', 'dislocation of', 'exclusion of no', 'no. of nodes in']


#### Build sklearn pipeline

In [8]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # classification model
            ('obj', obj)

 ])


#### Run Pipeline

In [9]:
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)


Fitting 2 folds for each of 4 candidates, totalling 8 fits


/Users/hjavedani/.pyenv/versions/3.7.8/lib/python3.7/site-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


0:	learn: 1.0964765	total: 52.1ms	remaining: 52.1s
1:	learn: 1.0940183	total: 52.7ms	remaining: 26.3s
2:	learn: 1.0925400	total: 53.2ms	remaining: 17.7s
0:	learn: 1.0964066	total: 52.2ms	remaining: 52.1s
3:	learn: 1.0910035	total: 53.8ms	remaining: 13.4s
1:	learn: 1.0938873	total: 52.9ms	remaining: 26.4s
4:	learn: 1.0888035	total: 54.3ms	remaining: 10.8s
2:	learn: 1.0917111	total: 53.5ms	remaining: 17.8s
5:	learn: 1.0867359	total: 54.8ms	remaining: 9.08s
3:	learn: 1.0894544	total: 54.2ms	remaining: 13.5s
6:	learn: 1.0837697	total: 55.3ms	remaining: 7.85s
4:	learn: 1.0873726	total: 54.7ms	remaining: 10.9s
7:	learn: 1.0823441	total: 55.8ms	remaining: 6.92s
5:	learn: 1.0856279	total: 55.1ms	remaining: 9.13s
8:	learn: 1.0803986	total: 56.2ms	remaining: 6.19s
6:	learn: 1.0825365	total: 55.6ms	remaining: 7.88s
9:	learn: 1.0788462	total: 56.7ms	remaining: 5.61s
7:	learn: 1.0801677	total: 56ms	remaining: 6.95s
10:	learn: 1.0776644	total: 57.1ms	remaining: 5.13s
8:	learn: 1.0782821	total: 56.5m

#### Check performance of the pipeline

In [10]:
print('F1 score : ')
print(f1_score(y_test,y_pred, average='weighted'))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))


F1 score : 
0.4419742365031119
Classification report : 
              precision    recall  f1-score   support

           0       0.46      0.57      0.51        21
           1       0.38      0.43      0.40        14
           2       0.57      0.29      0.38        14

    accuracy                           0.45        49
   macro avg       0.47      0.43      0.43        49
weighted avg       0.47      0.45      0.44        49

Confusion matrix : 
[[12  7  2]
 [ 7  6  1]
 [ 7  3  4]]
